<font color="#5642C2" size="5">Excersise 3 in Natural Language Processing</font>
<br>
<font color="#5642C2" size="4">Topic Modeling</font>
<br>
<font color="#5642C2" size="3">Author:</font><font color="#ADADAD" size="3">Angeliki Mylonaki</font>
<br>
<font color="#5642C2" size="3">Github:</font> https://github.com/geloumil/Topic-Modelling-and-NER.git

In [2]:
import glob
import os
import codecs

def getData(regex):

    file_list=[]
    trainFiles = glob.glob(os.path.join(".", regex)) #make list of paths

    for fd in trainFiles:
        with codecs.open(fd, encoding='utf-8', errors='ignore') as fp:
            file_list.append(fp.read().encode("utf-8"))
            
    return file_list

documents=getData("./nipstxt/nips*/*")
print type(documents[0])

<type 'str'>


<font color="#5642C2" size="3">Mining Documents</font>

In [56]:
from nltk import corpus
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re



def mineDocs(documents):
    
    porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    minedList=[]
    
    stopWords = corpus.stopwords.words("english")
    documents=[s.translate(None, string.punctuation) for s in documents]
    documents=[s.translate(None, "\x7f") for s in documents]   
    for doc in documents:
         
        #splitting document into sentences
        sentences = sent_tokenize(doc)

        for sentence in sentences:
            #removing digits
            sentence=sentence.translate(None, string.digits)
            
            #converting to lowercase
            sentence=sentence.lower()
            
            #removing stopwords
            sentence=' '.join([word for word in sentence.split() if word not in stopWords])
            
            #removing single letter words
            sentence=' '.join([word for word in sentence.split() if len(word) >1 ])
            
            #getting words of phrase
            words=word_tokenize(sentence)
            
            #stemming and lemmatizing
            words=[porter_stemmer.stem(word) for word in words]
            words=[wordnet_lemmatizer.lemmatize(word) for word in words]
                      
            minedList.append(sentence)
            
    return minedList



mdocs=mineDocs(documents)
mdocs_tokenized=[doc.split() for doc in mdocs]

<font color="#5642C2" size="3">Vectorizing,TF-IDF </font>

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(stop_words="english", binary="true")
m_docs_vector = count_vect.fit_transform(mdocs)

tfidf_transformer = TfidfTransformer(use_idf=False).fit(m_docs_vector)

<font color="#5642C2" size="3">Creating Document-term Matrix</font>

In [58]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(mdocs_tokenized)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in mdocs_tokenized]

<font color="#5642C2" size="3">LDA</font>

In [59]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
from operator import itemgetter

num_topics=[10,50,20]
alpha=["auto","asymmetric","symmetric"]

# Running and Trainign LDA model on the document term matrix.
for topicNo in num_topics:
    for a in alpha:
        print "\n\nFor numberOfTopics=",topicNo," and a= ",a
        ldamodel = Lda(corpus=doc_term_matrix, num_topics=topicNo, id2word = dictionary,alpha=a)
        print ldamodel.print_topics(num_topics=topicNo, num_words=5)




For numberOfTopics= 10  and a=  auto
[(0, u'0.006*"set" + 0.006*"network" + 0.005*"learning" + 0.005*"neural" + 0.004*"networks"'), (1, u'0.005*"function" + 0.005*"one" + 0.005*"time" + 0.004*"neural" + 0.004*"model"'), (2, u'0.008*"learning" + 0.006*"data" + 0.006*"model" + 0.005*"network" + 0.005*"using"'), (3, u'0.006*"model" + 0.005*"neural" + 0.005*"network" + 0.004*"figure" + 0.004*"learning"'), (4, u'0.008*"learning" + 0.007*"network" + 0.007*"model" + 0.006*"figure" + 0.006*"neural"'), (5, u'0.007*"learning" + 0.006*"figure" + 0.005*"network" + 0.004*"function" + 0.004*"model"'), (6, u'0.008*"network" + 0.006*"learning" + 0.006*"model" + 0.005*"function" + 0.005*"neural"'), (7, u'0.006*"learning" + 0.006*"network" + 0.006*"function" + 0.005*"model" + 0.005*"neural"'), (8, u'0.008*"network" + 0.006*"input" + 0.006*"learning" + 0.005*"networks" + 0.005*"model"'), (9, u'0.010*"network" + 0.007*"neural" + 0.006*"model" + 0.006*"using" + 0.005*"learning"')]


For numberOfTopics= 1

[(0, u'0.007*"learning" + 0.005*"model" + 0.005*"neural" + 0.004*"network" + 0.004*"two"'), (1, u'0.006*"learning" + 0.005*"function" + 0.005*"network" + 0.005*"model" + 0.005*"used"'), (2, u'0.008*"model" + 0.006*"figure" + 0.006*"learning" + 0.005*"network" + 0.004*"algorithm"'), (3, u'0.007*"network" + 0.006*"function" + 0.006*"model" + 0.005*"learning" + 0.005*"output"'), (4, u'0.009*"network" + 0.007*"learning" + 0.006*"training" + 0.005*"function" + 0.005*"one"'), (5, u'0.008*"network" + 0.007*"model" + 0.006*"neural" + 0.005*"learning" + 0.005*"set"'), (6, u'0.007*"model" + 0.005*"neural" + 0.005*"network" + 0.005*"function" + 0.005*"figure"'), (7, u'0.008*"model" + 0.006*"neural" + 0.005*"figure" + 0.005*"learning" + 0.004*"network"'), (8, u'0.007*"learning" + 0.007*"model" + 0.006*"network" + 0.006*"function" + 0.005*"data"'), (9, u'0.007*"input" + 0.007*"network" + 0.005*"neural" + 0.005*"set" + 0.005*"time"'), (10, u'0.007*"model" + 0.005*"learning" + 0.005*"network" + 0.004

[(0, u'0.008*"network" + 0.008*"model" + 0.007*"learning" + 0.005*"function" + 0.005*"error"'), (1, u'0.006*"network" + 0.006*"model" + 0.005*"learning" + 0.005*"neural" + 0.004*"figure"'), (2, u'0.007*"network" + 0.006*"learning" + 0.005*"function" + 0.005*"neural" + 0.005*"figure"'), (3, u'0.008*"network" + 0.007*"learning" + 0.005*"input" + 0.005*"function" + 0.005*"model"'), (4, u'0.006*"data" + 0.006*"neural" + 0.005*"network" + 0.005*"learning" + 0.005*"input"'), (5, u'0.007*"model" + 0.006*"network" + 0.006*"learning" + 0.005*"data" + 0.005*"one"'), (6, u'0.006*"neural" + 0.006*"learning" + 0.006*"network" + 0.005*"figure" + 0.005*"input"'), (7, u'0.006*"network" + 0.006*"learning" + 0.006*"neural" + 0.005*"set" + 0.005*"training"'), (8, u'0.007*"network" + 0.005*"neural" + 0.005*"data" + 0.005*"set" + 0.005*"training"'), (9, u'0.008*"network" + 0.007*"neural" + 0.007*"input" + 0.006*"networks" + 0.005*"model"'), (10, u'0.007*"learning" + 0.007*"one" + 0.006*"model" + 0.005*"fig

<font color="#5642C2" size="3">Visualizing Results</font>

In [66]:
#printing most representative topic for the 5 fist documents        
print "Best fitting topic for each Document"
print "------------------------------------"
for idx,doc in enumerate(doc_term_matrix[:5]):
    print "Document ",idx, "topic: ",\
    max(ldamodel.get_document_topics(doc),key=lambda item:item[1])[0]
print "......."

import pyLDAvis
import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

Best fitting topic for each Document
------------------------------------
Document  0 topic:  4
Document  1 topic:  12
Document  2 topic:  10
Document  3 topic:  9
Document  4 topic:  17
.......


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      22.471067        1       1 -0.003331 -0.005854
9       8.992485        1       2  0.004992 -0.001646
13      8.179040        1       3  0.004596 -0.002446
0       7.309473        1       4 -0.002223 -0.003707
3       6.804013        1       5 -0.001964 -0.002916
6       6.228804        1       6  0.003014 -0.003016
10      6.057999        1       7  0.005481  0.000530
15      5.769255        1       8 -0.004723 -0.002591
17      5.044121        1       9 -0.005252 -0.003384
2       4.788738        1      10  0.009948 -0.000399
14      4.643358        1      11  0.002428 -0.000729
11      2.908691        1      12 -0.000219 -0.000198
12      2.493188        1      13 -0.004762  0.000055
5       1.944224        1      14 -0.002719  0.000322
8       1.918503        1      15  0.002289  0.003301
1       1.343053        1      16  0.001358  0.002984
7       1.250257        1      17 -0.002221  0.001636
18      0.972051        1      18 -0.001479  0.005775
19      0.734504        1      19 -0.002956  0.003854
16      0.147183        1      20 -0.002259  0.008429, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
97339  Default  16964.000000     learning  16964.000000  30.0000  30.0000
96195  Default  18646.000000      network  18646.000000  29.0000  29.0000
77294  Default  15049.000000        model  15049.000000  28.0000  28.0000
15074  Default  10839.000000          set  10839.000000  27.0000  27.0000
62428  Default  14949.000000       neural  14949.000000  26.0000  26.0000
93708  Default  10932.000000     training  10932.000000  25.0000  25.0000
13923  Default  12936.000000         data  12936.000000  24.0000  24.0000
54096  Default  11390.000000          one  11390.000000  23.0000  23.0000
87895  Default  11803.000000     networks  11803.000000  22.0000  22.0000
16181  Default  12242.000000     function  12242.000000  21.0000  21.0000
46200  Default  10642.000000        using  10642.000000  20.0000  20.0000
41825  Default  13070.000000        input  13070.000000  19.0000  19.0000
10637  Default  12411.000000       figure  12411.000000  18.0000  18.0000
44270  Default   9436.000000          two   9436.000000  17.0000  17.0000
2443   Default   8656.000000    algorithm   8656.000000  16.0000  16.0000
40405  Default   9922.000000         time   9922.000000  15.0000  15.0000
81921  Default   7954.000000       system   7954.000000  14.0000  14.0000
18494  Default   9312.000000         used   9312.000000  13.0000  13.0000
4795   Default   8277.000000       output   8277.000000  12.0000  12.0000
67925  Default   8171.000000       number   8171.000000  11.0000  11.0000
81321  Default   7672.000000        error   7672.000000  10.0000  10.0000
24791  Default   7047.000000        units   7047.000000   9.0000   9.0000
15430  Default   7354.000000      results   7354.000000   8.0000   8.0000
80047  Default   6876.000000        given   6876.000000   7.0000   7.0000
45839  Default   6383.000000        state   6383.000000   6.0000   6.0000
60646  Default   5435.000000  performance   5435.000000   5.0000   5.0000
3242   Default   6547.000000      problem   6547.000000   4.0000   4.0000
54991  Default   6590.000000         also   6590.000000   3.0000   3.0000
26988  Default   6024.000000       models   6024.000000   2.0000   2.0000
67755  Default   6964.000000  information   6964.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
87895  Topic20     17.900055     networks  11803.039062   0.0299  -5.4477
83212  Topic20      4.079691          way   1828.500000   0.4160  -6.9265
24791  Topic20     11.383225        units   7047.023926   0.0930  -5.9004
13375  Topic20      8.085399          new   4568.888184   0.1843  -6.2424
77294  Topic20     20.153534   